In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import os
from rasterio.sample import sample_gen
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
basedir = 'D:/source/statsandmodels/biodiversity'
sp = 'Protonotaria_citrea'
year = '2017'
spyear = sp + '_' + year

In [3]:
# Paths
outputdir = os.path.join(basedir,'ppp_paramsoutput')
os.makedirs(outputdir, exist_ok=True)
raster_path = os.path.join(basedir,f"params_{year}.tif")
presence_csv = os.path.join(basedir,'gbif_species_csvs',f"{spyear}.csv")
output_path = os.path.join(outputdir,f"pppparam_{spyear}.tif")

In [4]:
# === Load Presence Points ===
presence_df = pd.read_csv(presence_csv)
gdf = gpd.GeoDataFrame(presence_df, geometry=gpd.points_from_xy(presence_df.Longitude, presence_df.Latitude), crs="EPSG:4326")

# === Load Raster ===
with rasterio.open(raster_path) as src:
    bands = [src.read(i + 1) for i in range(src.count)]  # list of arrays
    transform = src.transform
    crs = src.crs
    bounds = src.bounds
    gdf = gdf.to_crs(crs)  # Reproject presence points to raster CRS
    raster_shape = src.shape

In [5]:
# === Extract Environmental Variables at Presence Points ===
coords = [(point.x, point.y) for point in gdf.geometry]

with rasterio.open(raster_path) as src:
    values = [list(x) for x in src.sample(coords)]
presence_data = pd.DataFrame(values, columns=[f'band_{i+1}' for i in range(src.count)])

presence_data['label'] = 1

# === Sample Background Points ===
num_background = 10000
xs = np.random.uniform(bounds.left, bounds.right, num_background)
ys = np.random.uniform(bounds.bottom, bounds.top, num_background)
bg_coords = list(zip(xs, ys))

with rasterio.open(raster_path) as src:
    bg_vals = [list(x) for x in src.sample(bg_coords)]

background_data = pd.DataFrame(bg_vals, columns=[f'band_{i+1}' for i in range(src.count)])
background_data = background_data.dropna()
background_data['label'] = 0

# === Combine and Train ===
data = pd.concat([presence_data, background_data], ignore_index=True)
data.to_parquet(f"{spyear}_training_data.parquet", index=False)
X = data.drop(columns='label')
y = data['label']

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score
import rasterio

# ------------------------
# === Helper: Batched Poisson Prediction ===
# ------------------------
def batched_predict_poisson(model, X, batch_size=100000):
    n = X.shape[0]
    y_intensity = np.full(n, np.nan)
    
    for i in range(0, n, batch_size):
        end = min(i + batch_size, n)
        batch = X[i:end]
        
        if np.isnan(batch).all():
            continue
        
        valid = ~np.isnan(batch).any(axis=1)
        if np.any(valid):
            preds = model.predict(batch[valid])
            y_intensity[i:end][valid] = preds

    y_intensity /= np.nanmax(y_intensity)
    return y_intensity

def batched_scale(X, scaler, batch_size=100000):
    n = X.shape[0]
    X_scaled = np.full_like(X, np.nan, dtype=np.float32)
    
    for i in range(0, n, batch_size):
        end = min(i + batch_size, n)
        batch = X[i:end]
        
        valid = ~np.isnan(batch).any(axis=1)
        if np.any(valid):
            X_scaled[i:end][valid] = scaler.transform(batch[valid])
    
    return X_scaled

# ------------------------
# === Prepare Presence and Background Data ===
# ------------------------
coords = [(point.x, point.y) for point in gdf.geometry]

with rasterio.open(raster_path) as src:
    values = [list(x) for x in src.sample(coords)]
presence_data = pd.DataFrame(values, columns=[f'band_{i+1}' for i in range(src.count)])
presence_data['label'] = 1

num_background = 10000
xs = np.random.uniform(bounds.left, bounds.right, num_background)
ys = np.random.uniform(bounds.bottom, bounds.top, num_background)
bg_coords = list(zip(xs, ys))

with rasterio.open(raster_path) as src:
    bg_vals = [list(x) for x in src.sample(bg_coords)]
background_data = pd.DataFrame(bg_vals, columns=[f'band_{i+1}' for i in range(src.count)])
background_data = background_data.dropna()
background_data['label'] = 0

# Combine
data = pd.concat([presence_data, background_data], ignore_index=True)
X = data.drop(columns='label').values
y = data['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ------------------------
# === 5-Fold Cross-Validation ===
# ------------------------
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

for train_idx, test_idx in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Presence = 1, background = small constant
    y_train_poisson = np.where(y_train==1, 1.0, 1e-4)
    
    model = PoissonRegressor(alpha=1e-4, max_iter=1000)
    model.fit(X_train, y_train_poisson)
    
    # Predict
    y_pred_intensity = model.predict(X_test)
    y_pred_prob = y_pred_intensity / y_pred_intensity.max()
    
    # Metrics
    y_pred_bin = (y_pred_prob >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_pred_prob)
    f1 = f1_score(y_test, y_pred_bin)
    precision = precision_score(y_test, y_pred_bin)
    recall = recall_score(y_test, y_pred_bin)
    
    metrics.append({'auc': auc, 'f1': f1, 'precision': precision, 'recall': recall})

# Average metrics
mean_metrics = {k: np.mean([m[k] for m in metrics]) for k in metrics[0].keys()}
print("5-Fold CV Mean Metrics:", mean_metrics)

# ------------------------
# === Train Final Model on Full Dataset ===
# ------------------------
y_poisson_full = np.where(y==1, 1.0, 1e-4)
final_model = PoissonRegressor(alpha=1e-4, max_iter=1000)
final_model.fit(X_scaled, y_poisson_full)

# ------------------------
# === Predict Across Raster ===
# ------------------------
stack = np.stack(bands, axis=-1)  # H x W x bands
h, w, b = stack.shape
X_raster = stack.reshape(-1, b)
X_raster_scaled = batched_scale(X_raster, scaler, batch_size=100000)


5-Fold CV Mean Metrics: {'auc': 0.9298875287538607, 'f1': 0.018124676464724813, 'precision': 0.45999999999999996, 'recall': 0.009259259259259259}


In [7]:
import numpy as np
import rasterio
from rasterio.windows import Window
import gc
import psutil
process = psutil.Process()
print(f"Memory before predict: {process.memory_info().rss / 1e6:.2f} MB")
print(f"X_batch shape: {X_batch.shape}, dtype: {X_batch.dtype}")
print(f"Model coef shape: {model.coef_.shape}, intercept shape: {np.shape(model.intercept_)}")

def predict_raster_in_chunks(model, scaler, raster_path, output_path, batch_size=1000):
    """
    Memory-safe raster prediction that works with multi-band rasters and arbitrary window sizes.
    Reads, scales, predicts, and writes one window at a time.
    """

    with rasterio.open(raster_path) as src:
        profile = src.profile.copy()
        profile.update(
            dtype='float32',
            count=1,
            compress='lzw',
            tiled=True,
            blockxsize=128,
            blockysize=128
        )

        with rasterio.open(output_path, 'w', **profile) as dst:
            for _, window in src.block_windows(1):
                data = src.read(indexes=list(range(1, src.count + 1)), window=window)
                n_bands, h, w = data.shape

                # (pixels, bands)
                X = np.moveaxis(data, 0, -1).reshape(-1, n_bands)

                preds = np.full(X.shape[0], np.nan, dtype=np.float32)

                valid = ~np.isnan(X).any(axis=1)

                if np.any(valid):
                    valid_idx = np.where(valid)[0]

                    for start in range(0, len(valid_idx), batch_size):
                        end = start + batch_size
                        batch_idx = valid_idx[start:end]
                        X_batch = scaler.transform(X[batch_idx])

                        # Replace this:
                        # preds_batch = model.predict(X_batch)

                        # With this manual GLM step (linear prediction only)
                        raw_pred = X_batch @ model.coef_.T + model.intercept_

                        # If Poisson, apply inverse link (exponential)
                        preds_batch = np.exp(raw_pred).ravel()

                        preds[batch_idx] = preds_batch.astype(np.float32)

                preds_2d = preds.reshape((h, w))
                dst.write(preds_2d, indexes=1, window=window)

                # Clean up
                del data, X, preds, preds_2d
                gc.collect()

    print(f"✅ Predictions written to {output_path}")



#suitability = batched_predict_poisson(final_model, X_raster_scaled, batch_size=100000)
#suitability = suitability.reshape(h, w)
output_path = "predictions_poisson_2017.tif"
predict_raster_in_chunks(final_model, scaler, raster_path, output_path)

MemoryError: Unable to allocate 1.07 MiB for an array with shape (10000, 14) and data type float64

In [ ]:
# Standardize predictors (important for MaxEnt / Poisson)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

# === Fit Poisson Point Process Model (MaxEnt equivalent) ===
# Assign y = 1 for presence, small constant for background
y_train_poisson = np.where(y_train==1, 1.0, 1e-4)

poisson_model = PoissonRegressor(alpha=1e-4, max_iter=1000)
poisson_model.fit(X_train, y_train_poisson)

# === Predict on Test Set ===
y_pred_intensity = poisson_model.predict(X_test)
# Normalize to [0,1] for probability-like output
y_pred_prob = y_pred_intensity / y_pred_intensity.max()

# Metrics
auc = roc_auc_score(y_test, y_pred_prob)
cm = confusion_matrix(y_test, (y_pred_prob >= 0.5).astype(int))
report = classification_report(y_test, (y_pred_prob >= 0.5).astype(int))

print("AUC:", auc)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

# === Predict Across the Raster ===
# Stack all bands into one array (H x W x bands)
stack = np.stack(bands, axis=-1)  # shape: (H, W, bands)
h, w, b = stack.shape

X_raster = stack.reshape(-1, b)
X_raster_scaled = scaler.transform(X_raster)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score
# ------------------------
# === 5-Fold Cross-Validation ===
# ------------------------
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

for train_idx, test_idx in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Presence = 1, background = small constant
    y_train_poisson = np.where(y_train==1, 1.0, 1e-4)
    
    model = PoissonRegressor(alpha=1e-4, max_iter=1000)
    model.fit(X_train, y_train_poisson)
    
    # Predict
    y_pred_intensity = model.predict(X_test)
    y_pred_prob = y_pred_intensity / y_pred_intensity.max()
    
    # Metrics
    y_pred_bin = (y_pred_prob >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_pred_prob)
    f1 = f1_score(y_test, y_pred_bin)
    precision = precision_score(y_test, y_pred_bin)
    recall = recall_score(y_test, y_pred_bin)
    
    metrics.append({'auc': auc, 'f1': f1, 'precision': precision, 'recall': recall})

# Average metrics
mean_metrics = {k: np.mean([m[k] for m in metrics]) for k in metrics[0].keys()}
print("5-Fold CV Mean Metrics:", mean_metrics)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

In [ ]:
# -----------------------------
# Step 6: Evaluate model
# -----------------------------

y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Optional: Feature importance
importances = model.feature_importances_
plt.figure(figsize=(10, 5))
plt.bar(range(len(importances)), importances)
plt.title("Alpha-Earth Embedding Feature Importance")
plt.xlabel("Embedding Dimension")
plt.ylabel("Importance")
plt.show()

In [ ]:
# Simple accuracy
acc = accuracy_score(y_test, y_pred)

# ROC AUC (better for presence/background models)
auc = roc_auc_score(y_test, y_prob)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Detailed classification report (precision, recall, F1)
report = classification_report(y_test, y_pred, digits=3)

print("Accuracy:", acc)
print("ROC AUC:", auc)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

In [ ]:
from sklearn.model_selection import cross_val_score

auc_scores = cross_val_score(
    model, X, y, cv=5, scoring="roc_auc"
)
print("Mean AUC:", auc_scores.mean())
print("All AUCs:", auc_scores)

In [ ]:
from sklearn.metrics import jaccard_score

# Binary classification example
# y_test: true labels
# y_pred: predicted labels (not probabilities)

mIoU = jaccard_score(y_test, y_pred)  # For binary case
print("mIoU (Jaccard Index):", mIoU)

In [ ]:
'''
| Metric           | What It Tells You                           | Best Use Case                            |
| ---------------- | ------------------------------------------- | ---------------------------------------- |
| Accuracy         | Overall correctness                         | Balanced classes                         |
| ROC AUC          | How well positives are ranked               | Imbalanced classes (presence/background) |
| Confusion Matrix | Distribution of prediction errors           | Diagnostic (where model fails)           |
| Precision        | How many predicted presences are correct    | When false positives are costly          |
| Recall           | How many real presences were found          | When missing presences is bad            |
| F1-score         | Balance between precision and recall        | General performance                      |
| Cross-val AUC    | Generalization across multiple folds        | More robust evaluation                   |
| mIoU             | Overlap between predicted and real presence | Spatially meaningful accuracy            |

'''